In [1]:
import tensorflow as tf

<h3 style='color:purple'>Create tf dataset from a list</h3>

In [3]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

<h3 style='color:purple'>Iterate through tf dataset</h3>

In [5]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


<h3 style='color:purple'>Iterate through elements as numpy elements</h3>

In [7]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


<h3 style='color:purple'>Iterate through first n elements in tf dataset</h3>

In [9]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


<h3 style='color:purple'>Filter sales numbers that are < 0</h3>

In [11]:
tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


<h3 style='color:purple'>Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->72 conversation rate</h3>

In [13]:
tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


<h3 style='color:purple'>Shuffe</h3>

In [15]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
1512
2304
2232
2232
2448


https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset

<h3 style='color:purple'>Batching</h3>

In [18]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1584 2232]
[1512 2448]
[2304 2232]


<h3 style='color:purple'>Perform all of the above operations in one shot</h3>

In [20]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 1584]
[2304 2448]
[2232 2232]


<h3>Images</h3>

In [22]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

In [23]:
image_count = len(images_ds)
image_count

130

In [24]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [25]:
for file in images_ds.take(3):
    print(file.numpy())

b'images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'images\\cat\\A cat appears to have caught the....jpg'


In [26]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images\\dog\\2nd pet dog tests positive for COVID-19....jpg'
b'images\\dog\\why dogs understand our body language....jpg'
b'images\\dog\\Dog Breed Chart....jpg'


In [27]:
class_names = ["cat","dog"]

In [28]:
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [29]:
len(train_ds)

104

In [30]:
len(test_ds)

26

In [31]:
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [32]:
get_label("images\\dog\\20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [33]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [34]:
img, label = process_image("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

array([[[155.      , 186.      , 215.      ],
        [156.      , 187.      , 216.      ],
        [158.      , 189.      , 218.      ],
        [160.0039  , 189.0039  , 219.0039  ],
        [161.0039  , 190.0039  , 220.0039  ],
        [162.      , 191.      , 221.      ],
        [166.      , 193.      , 222.      ],
        [167.      , 194.      , 223.      ],
        [168.      , 195.      , 224.      ],
        [169.      , 196.      , 225.      ],
        [170.      , 197.      , 224.      ],
        [170.0039  , 197.0039  , 224.0039  ],
        [172.      , 199.      , 226.      ],
        [173.0039  , 199.0039  , 224.0039  ],
        [174.0039  , 200.0039  , 225.0039  ],
        [175.0039  , 201.0039  , 226.0039  ],
        [176.      , 202.      , 227.      ],
        [177.0039  , 203.0039  , 228.0039  ],
        [177.0039  , 203.0039  , 228.0039  ],
        [179.0039  , 203.0039  , 227.0039  ],
        [180.0039  , 204.0039  , 228.0039  ],
        [180.0039  , 204.0039  , 2

In [35]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [36]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[237.       237.       237.      ]
  [237.       237.       237.      ]
  [237.       237.       237.      ]
  ...
  [204.3274   152.3274   105.32739 ]
  [196.55151  152.34058  107.41089 ]
  [206.70605  162.70605  117.706055]]

 [[237.       237.       237.      ]
  [237.       237.       237.      ]
  [237.       237.       237.      ]
  ...
  [197.75977  146.75977  101.759766]
  [203.83618  156.72632  114.75049 ]
  [208.47388  159.948    116.71094 ]]

 [[237.       237.       237.      ]
  [237.       237.       237.      ]
  [237.       237.       237.      ]
  ...
  [204.38525  157.32275  115.291504]
  [209.96338  158.26147  119.13086 ]
  [200.25073  148.28345  108.19116 ]]

 ...

 [[254.95312  254.95312  254.95312 ]
  [253.       253.       253.      ]
  [255.       255.       255.      ]
  ...
  [255.       255.       255.      ]
  [255.       255.       255.      ]
  [255.       255.       255.      ]]

 [[254.95312  254.95312  254.95312 ]
  [253.       253.   

In [37]:
def scale(image, label):
    return image/255, label

In [38]:
train_ds = train_ds.map(scale)

In [39]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.6948223  0.43556985 0.12429535]
****Label:  b'cat'
****Image:  [0.27873775 0.46697304 0.7571691 ]
****Label:  b'dog'
****Image:  [0.37892157 0.5679611  0.6220588 ]
****Label:  b'cat'
****Image:  [0.8019167 0.8019167 0.8019167]
****Label:  b'cat'
****Image:  [0.38673407 0.51222426 0.66311276]
****Label:  b'cat'
